In [255]:
-- Gloria Budiman
-- EE569 HW4
-- 6528-1836-50
-- 11/25/2016

require 'nn'
require 'optim'

torch.manualSeed(0)
torch.setnumthreads(4)

In [256]:
function random_translate(obj) 
    n = torch.Tensor(1, 40, 40):zero()
    n[{ {1}, {5, 36}, {5, 36}}] = obj
    
    rand_x = torch.random(-6, 6)
    rand_y = torch.random(-6, 6)
    
    n = image.translate(n, rand_x, rand_y)
    n = image.crop(n, 4, 4, 36, 36)
    return n
end

In [257]:
function normalize(input, n_channels)
    local mean = {}
    local stdev = {}
    
    for channel = 1, n_channels do
        mean[channel] = input.data[{ {}, {channel}, {}, {} }]:mean()
        stdev[channel] = input.data[{ {}, {channel}, {}, {} }]:std()
        
        print('Channel ' .. channel .. ' mean: ' .. mean[channel] .. ' stdev: ' .. stdev[channel])
        
        input.data[{ {}, {channel}, {}, {} }]:add(-mean[channel])
        input.data[{ {}, {channel}, {}, {} }]:div(stdev[channel])
    end
end

In [258]:
function load_data()    
    train = torch.load('mnist-p1b-train.t7')
    test = torch.load('mnist-p1b-test.t7')
    
    n_train = train.data:size()[1]
    n_test = test.data:size()[1]
    
    for i = 1, n_test do
        test.data[i] = random_translate(test.data[i])
    end
    
    train.data = train.data:double()
    test.data = test.data:double()
end

In [259]:
function setup() 
    classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 }
    geometry = { 32, 32 }

    net = nn.Sequential()

    net:add(nn.SpatialConvolution(1, 6, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.SpatialConvolution(6, 16, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    -- net:add(nn.View(16*5*5))
    net:add(nn.View(16*5*5))
    net:add(nn.Linear(16*5*5, 120))
    net:add(nn.ReLU())
    net:add(nn.Linear(120, 84))
    net:add(nn.ReLU())
    net:add(nn.Linear(84, #classes))
    net:add(nn.LogSoftMax())
    
    parameters, gradParameters = net:getParameters()
    criterion = nn.ClassNLLCriterion()
    confusion = optim.ConfusionMatrix(classes)
end

In [260]:
function exec_training(obj, n_channels, current_epoch) 
    confusion:zero()
    n_randomization = 8
    
    for t = 1, n_train, batch_size do
        local limit = math.min(t + batch_size - 1, n_train)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size * n_randomization, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size * n_randomization)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
            
            for j = 1, n_randomization - 1 do
                inputs[k] = random_translate(input)
                targets[k] = target
                k = k + 1
            end
        end
        
        local feval = function(x)
            collectgarbage()
            
            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()
            
            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)
            
            for i = 1, limited_batch_size * n_randomization do
                confusion:add(outputs[i], targets[i])
            end
            
            return f, gradParameters
        end
        
        sgd_state = sgd_state or {
            learningRate = 0.03,
            learningRateDecay = 1e-7,
            momentum = 0.5
        }
        optim.sgd(feval, parameters, sgd_state)
    end
    
    confusion:updateValids()
    return confusion.averageValid
end

In [261]:
function exec_testing(obj, n_channels, current_epoch)
    confusion:zero()
    
    for t = 1, n_test, batch_size do
        local limit = math.min(t + batch_size - 1, n_test)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local preds = net:forward(inputs)
        
        for i = 1, limited_batch_size do
            confusion:add(preds[i], targets[i])
        end
    end 
    
    confusion:updateValids()
    return confusion.averageValid
end

In [262]:
load_data()
normalize(train, 1)
normalize(test, 1)

Channel 1 mean: 25.509416422526 stdev: 70.180423838273	


Channel 1 mean: 25.782047851562 stdev: 70.645784650166	


In [263]:
setup()
n_epoch = 32
batch_size = 256
local last_train_conf
local last_test_conf
for epoch = 1, n_epoch do
    local acc_train = exec_training(train, 1, epoch)
    
    if (epoch == n_epoch) then
        last_train_conf = confusion:__tostring__()
    end
    
    local acc_test = exec_testing(test, 1, epoch)
    
    if (epoch == n_epoch) then
        last_test_conf = confusion:__tostring__()
    end
    
    io.write(string.format('Epoch %3d: %.4f%% | %.4f%%\n', epoch, acc_train * 100, acc_test * 100))
end

print(last_train_conf)
print(last_test_conf)

Epoch   1: 15.8035% | 22.5762%


Epoch   2: 43.1415% | 58.3297%


Epoch   3: 79.9997% | 78.9908%


Epoch   4: 89.0017% | 89.4515%


Epoch   5: 91.8014% | 92.2622%


...

Epoch  28: 97.6492% | 97.7469%


Epoch  29: 97.7066% | 97.7531%


Epoch  30: 97.7104% | 97.6964%


Epoch  31: 97.7697% | 97.8106%


Epoch  32: 97.8287% | 97.9240%
ConfusionMatrix:
[[   46800      19      74      11      23      56     199      17     113      72]   98.768% 	[class: 1]
 [      11   53406     168      45      49      14      22      99      96      26]   99.017% 	[class: 2]
 [      68     111   46567     262      27      24      34     256     276      39]   97.698% 	[class: 3]
 [      26      21     366   47615       0     312       1     242     304     161]   97.078% 	[class: 4]
 [      26      39      42      17   45862       2     106      84      77     481]   98.130% 	[class: 5]
 [      58      21      25     260      34   42361     177      28     277     127]   97.678% 	[class: 6]
 [     158      52      40       2      97     142   46724       0     123       6]   98.690% 	[class: 7]
 [      30     110     300     126     134      10       0   49047      71     292]   97.859% 	[class: 8]
 [     133     163     176     244     120     270     165      63   45232     242]   96.633% 	[class: 9